In [0]:
from pyspark.sql.types import * 
from pyspark.sql.functions import *

In [0]:
df = spark.read.format('csv').option('inferSchema',True).option('header',True).load('/Volumes/workspace/landing/datafiles/BigMartSales.csv')

In [0]:
df.display()

### Union and Union by name

### Preparing Dataframes

In [0]:
schema_one = "id String, name string"
data_one = [("1", "John"), ("2", "Mary")]

df_one = spark.createDataFrame(data_one, schema_one)

data2 = [("3", "rahul"), ("4", "jas")]
schema2 = "id STRING, name STRING"

df_two = spark.createDataFrame(data2, schema2)


In [0]:
df_one.display()
df_two.display()

In [0]:
df_one.union(df_two).display()

In [0]:
schema_one = "id String, name string"
data_one = [("1", "John"), ("2", "Mary")]

df_one = spark.createDataFrame(data_one, schema_one)
df_one.display()

In [0]:
df_one.union(df_two).display()